In [2]:
# 지정된 버전의 라이브러리들이 설치되어 있는지 확인
# !pip install transformers==4.35.0 accelerate==0.24.0 peft==0.6.0 \
# bitsandbytes==0.41.0 trl==0.7.4 datasets torch
# !uv pip install transformers==4.35.0 accelerate==0.24.0 peft==0.6.0 \
# bitsandbytes==0.41.0 trl==0.7.4 datasets torch

In [ ]:
# 데이터셋과 훈련 관련 구성 요소 불러오기
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, BitsAndBytesConfig
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

# 샘플 파인튜닝 데이터 생성
sample_data = {
    'text': [
        "질문: 파이썬에서 리스트란 무엇인가요?\n답변: 리스트는 순서가 있는 데이터 모음으로, 대괄호로 표현합니다.",
        "질문: 머신러닝이란 무엇인가요?\n답변: 머신러닝은 데이터로부터 패턴을 학습하여 예측하는 AI 기술입니다.",
        "질문: 파이썬에서 함수를 정의하는 방법은?\n답변: def 키워드를 사용하여 함수를 정의할 수 있습니다.",
        "질문: 딥러닝과 머신러닝의 차이점은?\n답변: 딥러닝은 인공신경망을 사용하는 머신러닝의 한 분야입니다.",
        "질문: 자연어 처리란 무엇인가요?\n답변: 자연어 처리는 컴퓨터가 인간의 언어를 이해하고 처리하는 기술입니다."
    ]
}

# CSV 파일 생성 및 데이터셋 로드
df = pd.DataFrame(sample_data)
csv_path = './finetune_data.csv'
df.to_csv(csv_path, index=False, encoding='utf-8')

tune_data = load_dataset("csv", data_files=csv_path)
print(f"파인튜닝 데이터셋 로드 완료: {len(tune_data['train'])} 샘플")

# 4비트 양자화 설정 (BitsAndBytesConfig)
# quantize_params = BitsAndBytesConfig(
#     load_in_4bit=True,                       # 4비트 로딩 활성화
#     bnb_4bit_compute_dtype=torch.float16,    # 연산용 dtype은 float16
#     bnb_4bit_quant_type='nf4',               # NF4 양자화 형식 사용
#     bnb_4bit_use_double_quant=False          # 이중 양자화 비활성화
# )
# 4비트 양자화 설정 (BitsAndBytesConfig) - CPU 환경에서는 비활성화
quantize_params = None  # CPU 환경에서는 양자화 비활성화
# LoRA 설정 (LoraConfig)
lora_params = LoraConfig(
    r=64,                                    # LoRA 랭크 차원
    lora_alpha=16,                           # LoRA 스케일링 계수
    lora_dropout=0.1,                        # LoRA 드롭아웃 확률
    target_modules=["q_proj", "v_proj"],     # 적용할 모듈 지정
    bias="none",                             # 바이어스 설정
    task_type="CAUSAL_LM"                    # 작업 유형
)

# 훈련 설정 구성 (TrainingArguments)
train_params = TrainingArguments(
    output_dir='./model_outputs',            # 출력 디렉터리
    num_train_epochs=3,                      # 훈련 에포크 수
    per_device_train_batch_size=2,           # 장치당 훈련 배치 크기 (메모리 절약)
    per_device_eval_batch_size=2,            # 장치당 평가 배치 크기
    gradient_accumulation_steps=4,           # 그레이디언트 누적 스텝 수
    optim="paged_adamw_32bit",               # 32비트 페이징 AdamW 옵티마이저
    learning_rate=2e-4,                      # 학습률 (LoRA에 적합하게 조정)
    weight_decay=0.01,                       # 가중치 감쇠
    warmup_ratio=0.03,                       # 워밍업 비율
    max_grad_norm=1.0,                       # 그레이디언트 클리핑
    group_by_length=True,                    # 길이별 그룹화
    lr_scheduler_type="cosine",              # 코사인 스케줄러
    fp16=True,                               # FP16 사용 (호환성)
    gradient_checkpointing=True,             # 그레이디언트 체크포인팅
    save_steps=50,                           # 저장 간격
    logging_steps=10,                        # 로깅 간격
    evaluation_strategy="no",                # 평가 전략
    save_total_limit=2,                      # 저장할 체크포인트 수
    report_to="none"                         # 리포트 비활성화
)

# 모델과 토크나이저 로드를 위한 대안 설정 (Llama-2가 접근 제한이 있는 경우)
try:
    # LLaMA-2 모델 시도
    model_name = 'meta-llama/Llama-2-7b-hf'
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantize_params,
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    print("LLaMA-2 모델 로드 성공!")
    
except Exception as e:
    print(f"LLaMA-2 로드 실패: {e}")
    print("대안 모델을 사용합니다...")
    
    # 대안: 공개 모델 사용
    from transformers import AutoTokenizer, AutoModelForCausalLM
    model_name = 'microsoft/DialoGPT-medium'
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantize_params,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("대안 모델 로드 성공!")

# SFTTrainer를 이용한 지도 학습 기반 파인 튜닝 설정
sft_trainer = SFTTrainer(
    model=model,                             # 모델
    args=train_params,                       # 훈련 파라미터
    train_dataset=tune_data['train'],        # 훈련 데이터셋
    tokenizer=tokenizer,                     # 토크나이저
    peft_config=lora_params,                 # PEFT 설정
    max_seq_length=512,                      # 시퀀스 최대 길이
    dataset_text_field='text',               # 텍스트 필드 지정
    packing=False                            # 패킹 비활성화 (안정성)
)

print("SFTTrainer 설정 완료!")
print(f"훈련 데이터 크기: {len(tune_data['train'])}")
print(f"배치 크기: {train_params.per_device_train_batch_size}")
print(f"누적 스텝: {train_params.gradient_accumulation_steps}")
print(f"유효 배치 크기: {train_params.per_device_train_batch_size * train_params.gradient_accumulation_steps}")

# 훈련 실행 준비
print("\n훈련을 시작하려면 다음 코드를 실행하세요:")
print("sft_trainer.train()")
print("\n훈련 완료 후 모델 저장:")
print("sft_trainer.model.save_pretrained('./fine_tuned_model')")
print("tokenizer.save_pretrained('./fine_tuned_model')")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

파인튜닝 데이터셋 로드 완료: 5 샘플


/Users/corazzon/codes/.venv/lib/python3.13/site-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or NPU devices or certain XPU devices (with IPEX).